### Inverse kinematics of `Panda` with collision detection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mujoco_py
from mujoco_parser import *
from util import *
np.set_printoptions(precision=2)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("Done.")

### Spawn `Panda` with objects and check collision while solving IK

In [ ]:
env = MuJoCoParserClass(name='Panda',rel_xml_path='../asset/panda/franka_panda_w_objs.xml',VERBOSE=False)
random_place_objects(
    env,prefix='obj_',n_place=10,x_range=[0.3,1.0],y_range=[-0.5,0.5],z_range=[1.01,1.01],obj_min_dist=0.15)
env.init_viewer(TERMINATE_GLFW=True,INITIALIZE_GLFW=True,window_width=0.5,window_height=0.5)
env.set_max_tick(max_tick=10000)

# Init IK
body_name = 'panda_eef'
q         = env.get_q_pos(q_pos_idxs=env.rev_joint_idxs)
p_EE      = env.get_p_body(body_name=body_name)
R_EE      = env.get_R_body(body_name=body_name)
p_trgt    = p_EE + np.array([0.5,0.0,-0.5])
R_trgt    = R_EE

# Buffers
err_list = np.zeros(env.max_tick)
q_list   = np.zeros((env.max_tick,env.n_rev_joint))
while env.IS_ALIVE():
    # Numerical IK
    dq,err = env.one_step_ik(body_name=body_name,p_trgt=p_trgt,R_trgt=R_trgt,th=3.0*np.pi/180.0)
    q = q + dq
    # FK
    env.forward(q_pos=q,q_pos_idxs=env.rev_joint_idxs)
    # Render
    env.add_marker(env.get_p_body(body_name),radius=0.02,color=np.array([0,1,0,0.5]))
    env.add_marker(p_trgt,radius=0.02,color=np.array([0,1,0,0.5])) 
    
    for c_idx in range(env.sim.data.ncon):
        contact    = env.sim.data.contact[c_idx]
        p_contact  = contact.pos
        bodyname1  = env.body_idx2name(env.sim.model.geom_bodyid[contact.geom1])
        bodyname2  = env.body_idx2name(env.sim.model.geom_bodyid[contact.geom2])
        label      = '%s-%s'%(bodyname1,bodyname2)
        env.add_marker(pos=p_contact,radius=0.1,color=np.array([1,0,0,0.5]),label=label)
    
    env.render(RENDER_ALWAYS=True)
    # Append
    q_list[env.tick-1,:] = env.get_q_pos(q_pos_idxs=env.rev_joint_idxs)
print ("Done.")

In [ ]:
env.terminate_viewer()